# Selected Economic Characteristics: Employment Status from the American Community Survey

**[Work in progress]**

This notebook downloads [selected economic characteristics (DP03)](https://data.census.gov/cedsci/table?tid=ACSDP5Y2018.DP03) from the American Community Survey 2018 5-Year Data.

Data source: [American Community Survey 5-Year Data 2018](https://www.census.gov/data/developers/data-sets/acs-5year.html)

Authors: Peter Rose (pwrose@ucsd.edu), Ilya Zaslavsky (zaslavsk@sdsc.edu)

In [1]:
import os
import pandas as pd
from pathlib import Path
import time

In [2]:
pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columsns

In [3]:
NEO4J_IMPORT = Path(os.getenv('NEO4J_IMPORT'))
print(NEO4J_IMPORT)

/Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-9f7418e6-ef5d-4a2d-ae16-29a5a6814849/installation-4.1.0/import


## Download selected variables

* [Selected economic characteristics for US](https://data.census.gov/cedsci/table?tid=ACSDP5Y2018.DP03)

* [List of variables as HTML](https://api.census.gov/data/2018/acs/acs5/profile/groups/DP03.html) or [JSON](https://api.census.gov/data/2018/acs/acs5/profile/groups/DP03/)

* [Description of variables](https://www2.census.gov/programs-surveys/acs/tech_docs/subject_definitions/2018_ACSSubjectDefinitions.pdf)

* [Example URLs for API](https://api.census.gov/data/2018/acs/acs5/profile/examples.html)

### Specify variables from DP03 group and assign property names

Names must follow the [Neo4j property naming conventions](https://neo4j.com/docs/getting-started/current/graphdb-concepts/#graphdb-naming-rules-and-recommendations).

In [4]:
variables = {# EMPLOYMENT STATUS
             'DP03_0001E': 'population16YearsAndOver',
             'DP03_0002E': 'population16YearsAndOverInLaborForce',
             'DP03_0002PE': 'population16YearsAndOverInLaborForcePct',
             'DP03_0003E': 'population16YearsAndOverInCivilianLaborForce',
             'DP03_0003PE': 'population16YearsAndOverInCivilianLaborForcePct',
             'DP03_0006E': 'population16YearsAndOverInArmedForces',
             'DP03_0006PE': 'population16YearsAndOverInArmedForcesPct',
             'DP03_0007E': 'population16YearsAndOverNotInLaborForce',
             'DP03_0007PE': 'population16YearsAndOverNotInLaborForcePct'
             #'DP03_0014E': 'ownChildrenOfTheHouseholderUnder6Years',
             #'DP03_0015E': 'ownChildrenOfTheHouseholderUnder6YearsAllParentsInLaborForce',
             #'DP03_0016E': 'ownChildrenOfTheHouseholder6To17Years',
             #'DP03_0017E': 'ownChildrenOfTheHouseholder6To17YearsAllParentsInLaborForce',
            }

In [5]:
fields = ",".join(variables.keys())

In [6]:
for v in variables.values():
    print('e.' + v + ' = toInteger(row.' + v + '),')

e.population16YearsAndOver = toInteger(row.population16YearsAndOver),
e.population16YearsAndOverInLaborForce = toInteger(row.population16YearsAndOverInLaborForce),
e.population16YearsAndOverInLaborForcePct = toInteger(row.population16YearsAndOverInLaborForcePct),
e.population16YearsAndOverInCivilianLaborForce = toInteger(row.population16YearsAndOverInCivilianLaborForce),
e.population16YearsAndOverInCivilianLaborForcePct = toInteger(row.population16YearsAndOverInCivilianLaborForcePct),
e.population16YearsAndOverInArmedForces = toInteger(row.population16YearsAndOverInArmedForces),
e.population16YearsAndOverInArmedForcesPct = toInteger(row.population16YearsAndOverInArmedForcesPct),
e.population16YearsAndOverNotInLaborForce = toInteger(row.population16YearsAndOverNotInLaborForce),
e.population16YearsAndOverNotInLaborForcePct = toInteger(row.population16YearsAndOverNotInLaborForcePct),


In [7]:
print(len(variables.keys()))

9


## Download county-level data using US Census API

In [8]:
url_county = f'https://api.census.gov/data/2018/acs/acs5/profile?get={fields}&for=county:*'

In [9]:
df = pd.read_json(url_county, dtype='str')
df.fillna('', inplace=True)
df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,DP03_0001E,DP03_0002E,DP03_0002PE,DP03_0003E,DP03_0003PE,DP03_0006E,DP03_0006PE,DP03_0007E,DP03_0007PE,state,county
1,36109,20200,55.9,20174,55.9,26,0.1,15909,44.1,28,151
2,9508,4951,52.1,4928,51.8,23,0.2,4557,47.9,28,111
3,6718,3402,50.6,3398,50.6,4,0.1,3316,49.4,28,019
4,18949,9923,52.4,9923,52.4,0,0.0,9026,47.6,28,057


##### Add column names

In [10]:
df = df[1:].copy() # skip first row of labels
columns = list(variables.values())
columns.append('stateFips')
columns.append('countyFips')
df.columns = columns

Remove Puerto Rico (stateFips = 72) to limit data to US States

TODO handle data for Puerto Rico (GeoNames represents Puerto Rico as a country)

In [11]:
df.query("stateFips != '72'", inplace=True)

Save list of state fips (required later to get tract data by state)

In [12]:
stateFips = list(df['stateFips'].unique())
stateFips.sort()
print(stateFips)

['01', '02', '04', '05', '06', '08', '09', '10', '11', '12', '13', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '53', '54', '55', '56']


In [13]:
df.head()

,population16YearsAndOver,population16YearsAndOverInLaborForce,population16YearsAndOverInLaborForcePct,population16YearsAndOverInCivilianLaborForce,population16YearsAndOverInCivilianLaborForcePct,population16YearsAndOverInArmedForces,population16YearsAndOverInArmedForcesPct,population16YearsAndOverNotInLaborForce,population16YearsAndOverNotInLaborForcePct,stateFips,countyFips
1,36109,20200,55.9,20174,55.9,26,0.1,15909,44.1,28,151
2,9508,4951,52.1,4928,51.8,23,0.2,4557,47.9,28,111
3,6718,3402,50.6,3398,50.6,4,0.1,3316,49.4,28,019
4,18949,9923,52.4,9923,52.4,0,0.0,9026,47.6,28,057
5,8404,3843,45.7,3843,45.7,0,0.0,4561,54.3,28,015


In [14]:
# Example data
df[(df['stateFips'] == '06') & (df['countyFips'] == '073')]

,population16YearsAndOver,population16YearsAndOverInLaborForce,population16YearsAndOverInLaborForcePct,population16YearsAndOverInCivilianLaborForce,population16YearsAndOverInCivilianLaborForcePct,population16YearsAndOverInArmedForces,population16YearsAndOverInArmedForcesPct,population16YearsAndOverNotInLaborForce,population16YearsAndOverNotInLaborForcePct,stateFips,countyFips
1869,2656740,1745186,65.7,1671892,62.9,73294,2.8,911554,34.3,06,073


In [15]:
df['source'] = 'American Community Survey 5 year'
df['aggregationLevel'] = 'Admin2'

### Save data

In [16]:
df.to_csv(NEO4J_IMPORT / "03a-USCensusDP03EmploymentAdmin2.csv", index=False)

## Download zip-level data using US Census API

In [17]:
url_zip = f'https://api.census.gov/data/2018/acs/acs5/profile?get={fields}&for=zip%20code%20tabulation%20area:*'

In [18]:
df = pd.read_json(url_zip, dtype='str')
df.fillna('', inplace=True)
df.head()

,0,1,2,3,4,5,6,7,8,9
0,DP03_0001E,DP03_0002E,DP03_0002PE,DP03_0003E,DP03_0003PE,DP03_0006E,DP03_0006PE,DP03_0007E,DP03_0007PE,zip code tabulation area
1,6999,3906,55.8,3906,55.8,0,0.0,3093,44.2,43964
2,40412,28375,70.2,28333,70.1,42,0.1,12037,29.8,28216
3,54703,38793,70.9,38793,70.9,0,0.0,15910,29.1,28277
4,20753,16058,77.4,16038,77.3,20,0.1,4695,22.6,28278


##### Add column names

In [19]:
df = df[1:].copy() # skip first row
columns = list(variables.values())
columns.append('postalCode')
df.columns = columns

In [20]:
df.head()

,population16YearsAndOver,population16YearsAndOverInLaborForce,population16YearsAndOverInLaborForcePct,population16YearsAndOverInCivilianLaborForce,population16YearsAndOverInCivilianLaborForcePct,population16YearsAndOverInArmedForces,population16YearsAndOverInArmedForcesPct,population16YearsAndOverNotInLaborForce,population16YearsAndOverNotInLaborForcePct,postalCode
1,6999,3906,55.8,3906,55.8,0,0.0,3093,44.2,43964
2,40412,28375,70.2,28333,70.1,42,0.1,12037,29.8,28216
3,54703,38793,70.9,38793,70.9,0,0.0,15910,29.1,28277
4,20753,16058,77.4,16038,77.3,20,0.1,4695,22.6,28278
5,23635,14333,60.6,12189,51.6,2144,9.1,9302,39.4,28303


In [21]:
# Example data
df.query("postalCode == '90210'")

,population16YearsAndOver,population16YearsAndOverInLaborForce,population16YearsAndOverInLaborForcePct,population16YearsAndOverInCivilianLaborForce,population16YearsAndOverInCivilianLaborForcePct,population16YearsAndOverInArmedForces,population16YearsAndOverInArmedForcesPct,population16YearsAndOverNotInLaborForce,population16YearsAndOverNotInLaborForcePct,postalCode
30897,16439,9181,55.8,9138,55.6,43,0.3,7258,44.2,90210


In [22]:
df['source'] = 'American Community Survey 5 year'
df['aggregationLevel'] = 'PostalCode'

### Save data

In [23]:
df.to_csv(NEO4J_IMPORT / "03a-USCensusDP03EmploymentZip.csv", index=False)

## Download tract-level data using US Census API
Tract-level data are only available by state, so we need to loop over all states.

In [24]:
def get_tract_data(state):
    url_tract = f'https://api.census.gov/data/2018/acs/acs5/profile?get={fields}&for=tract:*&in=state:{state}'
    df = pd.read_json(url_tract, dtype='str')
    time.sleep(1)
    # skip first row of labels
    df = df[1:].copy()
    # Add column names
    columns = list(variables.values())
    columns.append('stateFips')
    columns.append('countyFips')
    columns.append('tract')
    df.columns = columns
    return df

In [ ]:
df = pd.concat((get_tract_data(state) for state in stateFips))
df.fillna('', inplace=True)

In [ ]:
df['tract'] = df['stateFips'] + df['countyFips'] + df['tract']

In [ ]:
df['source'] = 'American Community Survey 5 year'
df['aggregationLevel'] = 'Tract'

In [ ]:
# Example data for San Diego County
df[(df['stateFips'] == '06') & (df['countyFips'] == '073')].head()

### Save data

In [ ]:
df.to_csv(NEO4J_IMPORT / "03a-USCensusDP03EmploymentTract.csv", index=False)

In [ ]:
df.shape